In [123]:
import json
import spacy
import en_core_web_lg
nlp = spacy.load("en_core_web_lg")
import string
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import classification_report, make_scorer
from random import shuffle
import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics
import scipy.stats

In [109]:
with open('run-on-test.json') as f:
    hold = json.load(f)

In [3]:
with open('4300-0.txt') as f:
    content = f.readlines()
content[:15]

['\ufeff\n',
 'The Project Gutenberg EBook of Ulysses, by James Joyce\n',
 '\n',
 'This eBook is for the use of anyone anywhere at no cost and with\n',
 'almost no restrictions whatsoever. You may copy it, give it away or\n',
 're-use it under the terms of the Project Gutenberg License included\n',
 'with this eBook or online at www.gutenberg.org\n',
 '\n',
 '\n',
 'Title: Ulysses\n',
 '\n',
 'Author: James Joyce\n',
 '\n',
 'Release Date: August 1, 2008 [EBook #4300]\n',
 'Last Updated: December 27, 2019\n']

In [4]:
len(content)

32861

In [6]:
with open('4300-0.txt') as f:
    content = f.read()
text = content.split("— I —\n\n\n[ 1 ]")[1]

In [7]:
text = text.replace('\n', ' ').replace('—', '').replace('_', '')
t1 = text[:1000000]
sents1 = []
doc = nlp(t1)
for sent in doc.sents:
    if sent.string.strip()[-1] in ['!', '.', '?'] and sent.string.strip()[0].istitle():
        sents1.append(sent.string.strip())
t2 = text[1000000:]
sents2 = []
doc = nlp(t2)
for sent in doc.sents:
    if sent.string.strip()[-1] in ['!', '.', '?'] and sent.string.strip()[0].istitle():
        sents2.append(sent.string.strip())
sents = sents1 + sents2

In [8]:
len(sents)

21069

In [9]:
sent2 = sents[: int(len(sents) * .70)]
sent3 = sents[int(len(sents) * .70):int(len(sents) * .95)]
sent1 = sents[int(len(sents) * .95):int(len(sents) * .975)]
sent4 = sents[int(len(sents) * .975):]

In [10]:
def decapitalize(text):
    res = [text.split(' ')[0].lower()]+text.split(' ')[1:]
    return ' '.join(res)

In [11]:
def depunctuate(text):
    res = text.split(' ')[:-1]+[text.split(' ')[-1].strip(string.punctuation)]
    return ' '.join(res)

In [12]:
def label(text):
    sent = []
    doc = nlp(text)
    for token in doc:
        if token in doc[:-1]:
            sent.append([token.text, False])
        else:
            sent.append([token.text, True])
    return sent

In [13]:
def label_p(text):
    sent = []
    doc = nlp(text)
    for token in doc:
        sent.append([token.text, False])
    return sent

In [14]:
dsent2 = [(decapitalize(x) if i % 4 == 1 else x) for i, x in enumerate(sent2)]
dp2 = [depunctuate(x) for x in dsent2]
lab2 = [label(x) for x in dp2]
labeled_2 = [x+y for x,y in zip(*[iter(lab2)]*2)]

In [15]:
dsent3 = [(decapitalize(x) if i % 6 == 1 else x) for i, x in enumerate(sent3)]
dp3 = [depunctuate(x) for x in dsent3]
lab3 = [label(x) for x in dp3]
labeled_3 = [x+y+z for x,y,z in zip(*[iter(lab3)]*3)]

In [16]:
dsent4 = [(decapitalize(x) if i % 8 == 1 else x) for i, x in enumerate(sent4)]
dp4 = [depunctuate(x) for x in dsent4]
lab4 = [label(x) for x in dp4]
labeled_4 = [x+y+z+e for x,y,z,e in zip(*[iter(lab4)]*4)]

In [17]:
labeled_1 = [label_p(x) for x in sent1]

In [20]:
data = labeled_1 + labeled_2+ labeled_3+labeled_3
shuffle(data)
len(data)

11411

In [21]:
def position(idx, range_len):
    if idx == 0:
        return "start"
    elif idx == range_len - 1:
        return "end"
    else:
        return "middle"

In [48]:
def sent2features(sent):
    tokens = []
    labels = []
    positions = []
    for i in range(len(sent)):
        tokens.append(sent[i][0])
        labels.append(sent[i][1])
        positions.append(position(i, len(sent)))
    text = ' '.join(tokens)
    pos = [token.pos_ for token in nlp(text)]
    res = list((zip(tokens, positions, pos)))
    return res

In [49]:
def sent2labels(sent):
    return [label for token, label in sent]

In [97]:
%%time
train, test = train_test_split(data, test_size=0.3, random_state=42)
train_tokens = [sent2features(s) for s in train]
train_labels = [sent2labels(s) for s in train]

test_tokens = [sent2features(s) for s in test]
test_labels = [sent2labels(s) for s in test]

CPU times: user 1min 7s, sys: 21.4 ms, total: 1min 7s
Wall time: 1min 7s


In [98]:
print(len(train_tokens)==len(train_labels))
print(len(test_tokens)==len(test_labels))

True
True


In [84]:
def classify(token):
    res = []
    for i in range(len(token)):
        res.append(False)     
    return res

In [85]:
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()
test_labels_bin = mlb.fit_transform(test_labels)

In [86]:
predicted = [classify(x) for  x in test_tokens]
print(classification_report(test_labels_bin, mlb.transform(predicted)))

              precision    recall  f1-score   support

           0       0.98      1.00      0.99      3368
           1       0.00      0.00      0.00      3257

   micro avg       0.98      0.51      0.67      6625
   macro avg       0.49      0.50      0.50      6625
weighted avg       0.50      0.51      0.50      6625
 samples avg       0.98      0.52      0.67      6625



/home/liudmyla/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [87]:
def word2features(sent, i):
    word = sent[i][0]
    postag = sent[i][2]
    
    features = {
        'bias': 1.0,
        'word.lower()': word.lower(),
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
        'postag': postag,
        'postag[:2]': postag[:2],        
    }
    if i > 0:
        word1 = sent[i-1][0]
        postag1 = sent[i-1][2]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
            '-1:postag': postag1,
            '-1:postag[:2]': postag1[:2],
        })
    else:
        features['BOS'] = True
        
    if i < len(sent)-1:
        word1 = sent[i+1][0]
        postag1 = sent[i+1][2]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
            '+1:postag': postag1,
            '+1:postag[:2]': postag1[:2],
        })
    else:
        features['EOS'] = True       
    
    return features

In [88]:
def sent2features2(sent):
    return [word2features(sent, i) for i in range(len(sent))]

In [99]:
%%time
X_train = [sent2features2(s) for s in train_tokens]
y_train = train_labels

X_test = [sent2features2(s) for s in test_tokens]
y_test = test_labels

CPU times: user 466 ms, sys: 11.9 ms, total: 478 ms
Wall time: 477 ms


In [100]:
def bool_to_string(labels):
    return [str(x) for x in labels]

In [101]:
y_train = [bool_to_string(x) for x in y_train]
y_test = [bool_to_string(x) for x in y_test]

In [102]:
%%time
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs', 
    c1=0.1, 
    c2=0.1, 
    max_iterations=100, 
    all_possible_transitions=True
)
crf.fit(X_train, y_train)

CPU times: user 6.67 s, sys: 3.73 ms, total: 6.67 s
Wall time: 6.67 s


CRF(algorithm='lbfgs', all_possible_states=None, all_possible_transitions=True,
    averaging=None, c=None, c1=0.1, c2=0.1, calibration_candidates=None,
    calibration_eta=None, calibration_max_trials=None, calibration_rate=None,
    calibration_samples=None, delta=None, epsilon=None, error_sensitive=None,
    gamma=None, keep_tempfiles=None, linesearch=None, max_iterations=100,
    max_linesearch=None, min_freq=None, model_filename=None, num_memories=None,
    pa_type=None, period=None, trainer_cls=None, variance=None, verbose=False)

In [103]:
labels = list(crf.classes_)

In [104]:
y_pred = crf.predict(X_test)
metrics.flat_f1_score(y_test, y_pred, 
                      average='weighted', labels=labels)

0.9700955208983115

In [117]:
print(metrics.flat_classification_report(
    y_test, y_pred, labels=labels, digits=3
))

              precision    recall  f1-score   support

       False      0.975     0.993     0.984     61023
        True      0.932     0.797     0.859      7586

    accuracy                          0.971     68609
   macro avg      0.954     0.895     0.921     68609
weighted avg      0.970     0.971     0.970     68609



In [124]:
%%time
# define fixed parameters and parameters to search
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs', 
    max_iterations=100, 
    all_possible_transitions=True
)
params_space = {
    'c1': scipy.stats.expon(scale=0.5),
    'c2': scipy.stats.expon(scale=0.05),
}

# use the same metric for evaluation
f1_scorer = make_scorer(metrics.flat_f1_score, 
                        average='weighted', labels=labels)

# search
rs = RandomizedSearchCV(crf, params_space, 
                        cv=3, 
                        verbose=1, 
                        n_jobs=-1, 
                        n_iter=50, 
                        scoring=f1_scorer)
rs.fit(X_train, y_train)

Fitting 3 folds for each of 50 candidates, totalling 150 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=-1)]: Done  38 tasks      | elapsed:  1.0min
[Parallel(n_jobs=-1)]: Done 150 out of 150 | elapsed:  3.6min finished


CPU times: user 1min 13s, sys: 4.6 s, total: 1min 18s
Wall time: 3min 42s


RandomizedSearchCV(cv=3, error_score=nan,
                   estimator=CRF(algorithm='lbfgs', all_possible_states=None,
                                 all_possible_transitions=True, averaging=None,
                                 c=None, c1=None, c2=None,
                                 calibration_candidates=None,
                                 calibration_eta=None,
                                 calibration_max_trials=None,
                                 calibration_rate=None,
                                 calibration_samples=None, delta=None,
                                 epsilon=None, error_sensitive=None, gamma=None,
                                 keep_...
                   iid='deprecated', n_iter=50, n_jobs=-1,
                   param_distributions={'c1': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f36db318880>,
                                        'c2': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f36db4151c0>},
                

In [125]:
print('best params:', rs.best_params_)
print('best CV score:', rs.best_score_)
print('model size: {:0.2f}M'.format(rs.best_estimator_.size_ / 1000000))

best params: {'c1': 0.4349752672864775, 'c2': 0.007900081891281253}
best CV score: 0.9684019811528968
model size: 0.67M


In [128]:
crf = rs.best_estimator_
y_pred = crf.predict(X_test)
print(metrics.flat_classification_report(
    y_test, y_pred, labels=sorted_labels, digits=3
))

              precision    recall  f1-score   support

       False      0.975     0.993     0.984     61023
        True      0.938     0.795     0.861      7586

    accuracy                          0.972     68609
   macro avg      0.956     0.894     0.922     68609
weighted avg      0.971     0.972     0.971     68609



In [130]:
from collections import Counter
def print_transitions(trans_features):
    for (label_from, label_to), weight in trans_features:
        print("%-6s -> %-7s %0.6f" % (label_from, label_to, weight))

print("Top likely transitions:")
print_transitions(Counter(crf.transition_features_).most_common(20))

print("\nTop unlikely transitions:")
print_transitions(Counter(crf.transition_features_).most_common()[-20:])

Top likely transitions:
False  -> False   0.949754
False  -> True    0.583411
True   -> False   -0.909286
True   -> True    -1.895375

Top unlikely transitions:
False  -> False   0.949754
False  -> True    0.583411
True   -> False   -0.909286
True   -> True    -1.895375


In [131]:
def print_state_features(state_features):
    for (attr, label), weight in state_features:
        print("%0.6f %-8s %s" % (weight, label, attr))    

print("Top positive:")
print_state_features(Counter(crf.state_features_).most_common(30))

print("\nTop negative:")
print_state_features(Counter(crf.state_features_).most_common()[-30:])

Top positive:
10.968408 True     word.lower():aiulella
9.670033 False    -1:word.lower():awfur
9.158217 True     +1:word.lower():bluebags
8.640642 True     +1:word.lower():incog
8.342930 False    word.lower():regards
8.306212 True     -1:word.lower():unaffectedly
8.282945 True     +1:word.lower():bluerobed
8.064775 True     +1:word.lower():hushaby
7.523097 False    word.lower():follows
7.484504 True     word.lower():recant
7.144471 True     word.lower():vatican
7.078424 True     +1:word.lower():oho
6.916210 False    word.lower():arming
6.888639 True     +1:word.lower():scowl
6.843199 True     -1:word.lower():gliding
6.801322 True     word.lower():bean
6.534484 True     +1:word.lower():tutti
6.512819 True     -1:word.lower():subdued
6.494463 True     word.lower():galoptious
6.486044 True     word.lower():casteele
6.335321 True     -1:word.lower():shoving
6.314640 True     word.lower():polity
6.309456 False    +1:word.lower():,
6.288268 True     -1:word.lower():hurricane
6.283466 True   

In [132]:
hold_tokens = [sent2features(s) for s in hold]
hold_tokens = [sent2features2(s) for s in hold_tokens]
hold_labels = [sent2labels(s) for s in hold]
hold_labels = [bool_to_string(x) for x in hold_labels]

In [133]:
hold_pred = crf.predict(hold_tokens)
metrics.flat_f1_score(hold_labels, hold_pred, 
                      average='weighted', labels=labels)

0.9698063618192042